In [1]:
import re
from time import sleep
from selenium import webdriver
from pymongo import MongoClient
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [2]:
client = MongoClient(host='localhost', port=27017)

db = client['melon']

In [3]:
# 크롬 드라이버 설정
chrome_options = webdriver.ChromeOptions()

#지정한 user-agent로 설정합니다.
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
chrome_options.add_argument('headless')
chrome_options.add_argument('user-agent=' + user_agent)
chrome_options.add_argument('--blink-settings=imagesEnabled=false') 
chrome_options.add_argument("start-maximized")
driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
url = 'https://www.melon.com/dj/themegenre/djthemegenre_list.htm'
driver.get(url)
driver.implicitly_wait(10)
for idx_tag in range(1,28):#1,28
    sleep(3)
    theme = driver.find_element(By.CSS_SELECTOR, f'#tab1 > li:nth-child({idx_tag}) > a > span').text
    if theme == 'ASMR' or theme == '라운지':
        continue
    driver.find_element(By.CSS_SELECTOR, f'#tab1 > li:nth-child({idx_tag}) > a > span').click()
    driver.implicitly_wait(10)
    for idx_page_1 in range(5,6):#1,11
        sleep(5)
        if idx_page_1 == 1:
            pass
        else:
            driver.find_element(By.CSS_SELECTOR, f'#pageObjNavgation > div > span > a:nth-child({idx_page_1})').click()
            sleep(3)
        for idx_pl in range(1,21):#1,21
            back_url = driver.current_url
            sleep(2)
            driver.find_element(By.CSS_SELECTOR, f'#djPlylstList > div > ul > li:nth-child({idx_pl}) > div.entry > div.info > a.ellipsis.album_name').click()
            sleep(2)
            summary = driver.find_element(By.CSS_SELECTOR, f'#d_video_summary > div').text
            if '팝' in summary  or '피아노' in summary or '재즈' in summary or '해외' in summary or 'pop'.casefold() in summary.casefold() or 'edm'.casefold() in summary.casefold() or 'asmr'.casefold() in summary.casefold():
                if '케이팝' in summary or 'k-pop'.casefold() in summary.casefold() or 'kpop'.casefold() in summary.casefold():
                    pass
                else:
                    driver.back()
                    continue
            tag = []
            for t in summary.replace('\n','').split(" "):
                if  t[0] == '#':
                    tag.append(t.replace('#',''))
            page = len(driver.find_elements(By.CSS_SELECTOR, f'#pageObjNavgation > div > span > a'))+1
            for idx_page_2 in range(1, page+1):
                sleep(1)
                if idx_page_2 == 1:
                    pass
                else:
                    driver.find_element(By.CSS_SELECTOR, f'#pageObjNavgation > div > span > a:nth-child({idx_page_2})').click()
                sleep(2)
                list_count = len(driver.find_elements(By.CSS_SELECTOR, f'#frm > div > table > tbody > tr'))
                pass_music = 0
                for idx_list in range(1, list_count+1):
                    driver.implicitly_wait(10)
                    title = driver.find_element(By.CSS_SELECTOR, f'#frm > div > table > tbody > tr:nth-child({idx_list}) > td:nth-child(5) > div > div > div.ellipsis.rank01 > span > a').text
                    singer = driver.find_element(By.CSS_SELECTOR, f'#frm > div > table > tbody > tr:nth-child({idx_list}) > td:nth-child(5) > div > div > div.ellipsis.rank02').text
                    if re.compile('[^ㄱ-ㅣ가-힣]').sub('', title+singer) == '':
                        pass_music += 1
                        continue
                    if db.song.find_one({'title':title,'singer':singer}):
                        db.song.update_one({'title':title,'singer':singer},{'$addToSet' : {'tag' : {'$each':tag}}})
                        continue
                    driver.find_element(By.CSS_SELECTOR, f'#frm > div > table > tbody > tr:nth-child({idx_list}) > td:nth-child(4) > div > a').click()
                    sleep(1)
                    genre = driver.find_element(By.CSS_SELECTOR, f'#downloadfrm > div > div > div.entry > div.meta > dl > dd:nth-child(6)').text
                    try:
                        lyrics = driver.find_element(By.CSS_SELECTOR, f'#d_video_summary').text
                    except:
                        driver.back()
                        pass_music += 1
                        if pass_music > 10:
                            break
                        else:
                            continue
                    info = {'title' : title , 'singer' : singer, 'genre' : genre, 'lyrics' : lyrics}
                    db.song.insert_one(info)
                    db.song.update_one(info,{'$addToSet' : {'tag' : {'$each':tag}}})
                    driver.back()
            driver.get(back_url)
    driver.get(url)
driver.quit()


C:\Users\min\AppData\Local\Temp\ipykernel_12168\424048449.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)
C:\Users\min\AppData\Local\Temp\ipykernel_12168\424048449.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('C:/Users/min/chromedriver.exe', chrome_options=chrome_options)


ElementNotInteractableException: Message: element not interactable
  (Session info: headless chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x011C6643]
	(No symbol) [0x0115BE21]
	(No symbol) [0x0105D960]
	(No symbol) [0x010923A7]
	(No symbol) [0x01086B13]
	(No symbol) [0x010AFD7C]
	(No symbol) [0x0108641F]
	(No symbol) [0x010B00D4]
	(No symbol) [0x010C6B09]
	(No symbol) [0x010AFB76]
	(No symbol) [0x010849C1]
	(No symbol) [0x01085E5D]
	GetHandleVerifier [0x0143A142+2497106]
	GetHandleVerifier [0x014685D3+2686691]
	GetHandleVerifier [0x0146BB9C+2700460]
	GetHandleVerifier [0x01273B10+635936]
	(No symbol) [0x01164A1F]
	(No symbol) [0x0116A418]
	(No symbol) [0x0116A505]
	(No symbol) [0x0117508B]
	BaseThreadInitThunk [0x75DA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77E27BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77E27B8E+238]


In [4]:
idx_tag

3

In [5]:
idx_page_1

5

In [6]:
idx_pl

1

In [7]:
page

5

In [8]:
idx_page_2

3

In [9]:
idx_list

20

In [69]:
aa = 'ssss'+'Honolulu Swing'

In [70]:
type(aa)

str

In [18]:
import re

In [87]:
'Honolulu Swing'.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")

'Honolulu Swing'

In [83]:
a

'ssssHonolulu Swing'

In [94]:
hangul = re.compile('[^ㄱ-ㅣ가-힣]')

In [ ]:
hangul = re.compile('[^ㄱ-ㅣ가-힣]')

In [100]:
re.compile('[^ㄱ-ㅣ가-힣]').sub('', aa)

''

In [95]:
result = hangul.sub('', aa)

In [56]:
result = hangul.sub('', aa)

In [97]:
re.compile('[^ㄱ-ㅣ가-힣]')

re.compile(r'[^ㄱ-ㅣ가-힣]', re.UNICODE)

In [76]:
hangul.sub('', aa)

'ssssHonolulu Swing'

In [99]:
if result=='':
    print('a')

a
